In [5]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
vocabSize = 20
featureSize = 10

In [3]:
W = torch.randn(vocabSizeize,featureSize)

In [4]:
W.shape

torch.Size([20, 10])

In [6]:
E = nn.Embedding.from_pretrained(W)

In [7]:
idx = torch.LongTensor([1,2,5])
data = E(idx)

In [8]:
data

tensor([[-1.1224, -0.6627,  1.1105, -1.3627,  0.4476,  1.0151, -1.8652,  0.5181,
          0.2833, -0.8137],
        [-0.0940, -0.4296,  0.3623,  0.6620,  0.0987, -0.0064, -1.5370,  0.9694,
         -0.3084, -2.2359],
        [ 1.3636, -0.8082,  0.5907,  0.1404,  0.3482, -0.2167, -0.8259, -1.5568,
          0.7147, -0.2751]])

In [9]:
E = nn.Embedding(vocabSize,featureSize)

In [10]:
data = E(idx)

In [11]:
data

tensor([[ 1.4579,  2.2441,  0.4309,  0.2483,  1.2862,  0.9223, -2.1559,  0.8018,
          1.2421,  0.8221],
        [-0.5996,  0.8886,  1.3948,  0.4322, -0.0161, -0.9566, -0.0526,  0.5827,
         -0.3309, -0.0928],
        [-1.4420, -0.2347,  0.0319, -0.1356,  1.1695,  0.5495,  0.8208, -0.2333,
         -0.4967,  0.5436]], grad_fn=<EmbeddingBackward>)

In [13]:
vocabSize = 20
featureSize = 10
idx = torch.LongTensor(np.arange(vocabSize))
E = nn.Embedding(vocabSize,featureSize)

In [14]:
data = E(idx)

In [15]:
sqLength = 5
hiddenSize = 2
numLayers = 1
batchSize = 4

In [16]:
inputs = data.view(batchSize,sqLength,featureSize)

In [17]:
inputs.shape

torch.Size([4, 5, 10])

In [27]:
rnn = nn.RNN(input_size=featureSize,hidden_size=hiddenSize,
             num_layers=numLayers,batch_first=True)

In [33]:
rnn = nn.GRU(input_size=featureSize,hidden_size=hiddenSize,
             num_layers=numLayers,batch_first=True)

In [38]:
rnn_lstm = nn.LSTM(input_size=featureSize,hidden_size=hiddenSize,
             num_layers=numLayers,batch_first=True)

In [47]:
rnn_bi = nn.RNN(input_size=featureSize,hidden_size=hiddenSize,
             num_layers=3,batch_first=True,bidirectional=True)

In [48]:
y_hat, h = rnn_bi(inputs)

In [49]:
y_hat.shape

torch.Size([4, 5, 4])

In [51]:
outReshaped = y_hat.view(4,5,2,2)
y_hat_fwd = outReshaped[:,:,0,:]
y_hat_bck = outReshaped[:,:,1,:]

In [52]:
y_hat_fwd.shape

torch.Size([4, 5, 2])

In [50]:
h.shape

torch.Size([6, 4, 2])

In [53]:
hReshaped = h.view(3,2,4,2)
h_fwd = hReshaped[:,0,:,:]
h_bck = hReshaped[:,1,:,:]

In [54]:
h_fwd.shape

torch.Size([3, 4, 2])

In [39]:
y_hat,h = rnn_lstm(inputs)

In [40]:
y_hat.shape

torch.Size([4, 5, 2])

In [41]:
h[0].shape,h[1].shape

(torch.Size([1, 4, 2]), torch.Size([1, 4, 2]))

In [34]:
y_hat,h = rnn(inputs)

In [35]:
y_hat.shape

torch.Size([4, 5, 2])

In [36]:
y_hat[1,3,1]

tensor(0.4634, grad_fn=<SelectBackward>)

In [37]:
h.shape

torch.Size([1, 4, 2])

In [58]:
class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(RNN,self).__init__()
        self.hidden_size = hidden_size
        self.E = nn.Embedding(input_size,hidden_size)
        self.rnn = nn.GRU(hidden_size,hidden_size)
        self.linear = nn.Linear(hidden_size,output_size)
        self.sigmoid = nn.Sigmoid()
    def forward(self,input,hidden):
        emb = self.E(input).view(1,1,-1)
        output,hidden = self.rnn(emb,hidden)
        outLinear = self.linear(output)
        outSig = self.sigmoid(outLinear)
        return outSig,hidden

In [59]:
hidden_size = 128
model = RNN(20,hidden_size,1)

In [60]:
h = torch.zeros(1,1,hidden_size)
inputs = torch.LongTensor([2,3,4,8])
for input in inputs:
    y,h = model(input,h)

In [61]:
y

tensor([[[0.4888]]], grad_fn=<SigmoidBackward>)